<h2>Adam Cisak i Adama Jędrzejowski</h2>
<h1>Notatnik z kodem sieci i wczytywania danych </h1>
( całym kodem któy był pisany poza Jupyterem )
<h4>Opis wyników jest w notatniku wyniki.html (ze względu na wygodę przeglądania), od którego zalecamy zacząć</h4>
<h5>Projekt dostępny również pod adresem: <a href = https://www.github.com/Yoshud/Kohonen_neural_network>www.github.com/Yoshud/Kohonen_neural_network</a></h5>

<h2>main.py </h2>
<h4>(plik zawierający sposób wczytywania danych)</h4>

In [3]:
import csv
import numpy as np
import pandas as pd

DATA_LABELS = {
    "age": 0,
    "gender": 1,
    "tb": 2,
    "db": 3,
    "alkphos": 4,
    "sgpt": 5,
    "sgot": 6,
    "tp": 7,
    "alp": 8,
    "ag": 9,
    "class": 10
}


def load_data():
    with open("data.csv", newline='') as data_csv:
        data_read = csv.reader(data_csv, delimiter=',', quotechar='|')
        a = np.array(list(list(data_read)))
        a_pd = pd.read_csv("data.csv", sep=",")
        a_pd = parser(a_pd)
        return a_pd.T


def parser(data):
    return pd.DataFrame([parse_gender(col) if it == DATA_LABELS["gender"] else parse_class(col) if it == DATA_LABELS[
        "class"] else normalize(col) for it, col in enumerate(np.array(data.T))])


def normalize(row):
    max_v = max(row)
    return list(map(lambda el, max_v=max_v: el / max_v, row))


def parse_gender(row):
    return list(map(lambda el: 0 if el == "Female" else 1, row))


def parse_class(row):
    return list(map(lambda el: abs(el - 2), row))  # '1' - zywy, '0' - martwy


<h2>neural_network.py</h2>
<h4>(Definicje sieci, SOM i MLP, gdyż początkowo projekt miał być dość uniwersalny, ale odeszliśmy od tego pomysłu)</h4>

In [4]:
import layer #plik widoczny niżej
from functools import reduce
from abc import ABC, abstractmethod #funkcje abstrakcyjne w Pythonie


class NeuralNetwork(ABC):
    # @abstractmethod

    def set_data(self, treining_set, test_set):
        self.treining_set = treining_set
        self.test_set = test_set

    @abstractmethod
    def set_structure(self, structure, attributes_in_data):
        pass


# sieć typu MLP bez algorytmu uczenia
class MultiLayerNetwork(NeuralNetwork):
    def __init__(self, structure, attributes_in_data):
        self.set_structure(structure, attributes_in_data)

    def set_structure(self, structure, attributes_in_data):
        self.layers = []

        def add_layer(input, output, self=self):
            self.layers.append(layer.Layer(input, output, expit))
            return output

        reduce(add_layer, structure, attributes_in_data)

    def compute(self, x):
        return reduce(lambda returned, layer: layer.compute(returned), self.layers, x)

#Sieć typu SOM, ma zaimplementowany algorytm uczenia jej używający
class Kohonen2DNetwork(NeuralNetwork):
    def __init__(self, structure, attributes_in_data):
        self.set_structure(structure, attributes_in_data)

    def set_structure(self, structure, attributes_in_data):
        self.layers = layer.Kohonen2DLayer(attributes_in_data, structure)

#
#     def compute(self, x):
#         return reduce(lambda returned, layer: layer.compute(returned), self.layers, x)


<h2>layer.py</h2>
<h4>(Definicje warstw sieci. Tu znajduje się dość znacząca część kodu sieci SOM)</h4>

In [5]:
import cupy as cp #biblioteka działająca jak numpy, tylko że wykonująca operacje na GPU na rdzenach CUDA
import numpy as np #najpopularniejsza pythonowa biblioteka do operacji matematycznych głównie macierzowych
from math import floor, sqrt
from functools import reduce


# klasa przydatna na wypadek chęci tworzenia sieci MLP
class Layer:
    def __init__(self, number_of_input, number_of_output, activation_fun):
        self.W = cp.random.rand(number_of_input, number_of_output)
        self.activation_fun = activation_fun

    def compute(self, x):
        return self.activation_fun(cp.dot(x, self.W))

    def set_structure(self, number_of_input, number_of_output):
        self.W = cp.random.rand(number_of_input, number_of_output)


class Kohonen2DLayer:
    def __init__(self, number_of_input, number_of_output):
        # self.distance_dependency_fun = distance_dependency_fun
        self.set_structure(number_of_input, number_of_output)

    def set_structure(self, number_of_input, number_of_output):
        self.W = cp.random.rand(number_of_input, number_of_output)
        positions = list([])
        floor_sqrt_output = int(floor(sqrt(number_of_output)))
        for number_x in range(0, floor_sqrt_output):
            for number_y in range(0, floor_sqrt_output):
                positions.append([number_x, number_y])
        for number_y in range(0, number_of_output - floor_sqrt_output ** 2):
            positions.append([floor_sqrt_output, number_y])
        self.positions = cp.array(positions)

    # licz produkt macierzowy, właściwie zbędna funkcja
    def compute(self, x):
        return cp.dot(x, self.W)

    # wyszukuje BMU - neuronu najsilniej odpowiadającego. Korzysta z określania odległości jako kąty
    def find_best_neuron_by_dot_product(self, x):
        neurons_output = cp.dot(x, self.W).transpose()
        output_max = neurons_output[0]
        it_max = 0
        for it, product in enumerate(neurons_output):
            if abs(product) > abs(output_max):
                it_max = it
                output_max = product
        return it_max

    # wyszukuje BMU - neuronu najsilniej odpowiadającego
    def find_best_neuron_by_cartesian_distance(self, x):
        W = self.W.transpose()
        # funkcja ta nie jest dokładnie dystansem kartezjanskim ale zachowuje jego monotoniczność
        return reduce(lambda max, pretender: pretender if pretender[0] > max[0] else max,
                      map(lambda w, it, x=x: (abs(cp.sum(w - x)), it), W, range(len(W))))[1]

    # zakłada prostokątną architekture sieci, i liczy odległość między neuronami w "przestrzeni neuronów"
    def cartesian_distance_between_neurons(self, it_1, it_2):  # dla sieci o topoligi prostokątnej
        dist_x = (self.positions[it_1][0] - self.positions[it_2][0])
        dist_y = (self.positions[it_1][1] - self.positions[it_2][1])
        return sqrt(dist_x ** 2 + dist_y ** 2)

    def distance_between_neurons_as_vectors(self, it_1, it_2):
        W_1 = self.W[it_1]
        W_2 = self.W[it_2]
        # return sqrt(reduce(lambda sum, w: sum + w ** 2, W_1 - W_2))
        return sqrt(cp.sum((W_1 - W_2) ** 2))

    def number_of_neurons(self):
        return len(self.W.transpose())


<h2>kohonen_network_learning.py</h2>
<h4>(Definicja algorytmu uczenia sieci typu SOM. Na tym etapie projektu kod jest dość specyficzny, ale do etapu 3 będziemy chcieli go uogólnić pozwalająć na modyfikacje zachowania algorytmu z zewnątrz, a nie poprzez zmiane wewnątrz implementacji)</h4>

In [6]:
import numpy as np
from scipy.stats import norm  # potrzebne do prostej i szybkiej implementacji funkcji Gaussa
import copy  # używane do głębokich kopii sieci


# funktor pozwalający na ustawienie i potem zawsze zwracanie wartości funkcji Gaussa dla nastaw i danego punktu,
# zawsze zwraca wartość z 1 w mean - służy do liczenia wpływu odległości
class Gauss:
    def __init__(self, mean, std_dev):
        self.rv = norm(loc=mean, scale=std_dev)
        self.max_rv = self.rv.pdf(mean)

    def val(self, x):
        return self.rv.pdf(x) / self.max_rv


# sieć SOM z funkcją Gaussa jako funkcją odległości i ustaloną funkcją dla learning rate, w późniejszych etapach projektu
# ulegnie refaktoryzacji dla większej elastyczności (możliwość wyboru poszczególnych funkcji i ich parametrów)
class KohonenNetworkGauss_nbh:
    def __init__(self, data, network, dist_0=20, alpha_0=0.1, if_by_dot_product=False):
        self.data = data
        self.network = network
        self.by_dot_product = if_by_dot_product
        self.dist_0 = dist_0
        self.a_0 = alpha_0

    # funcja zwracająca indeksy danych w losowej kolejności by móc później "karmić" sieć danymi w losowej kolejności
    def randomize_data_indexes(self):
        one = list(range(0, len(self.data)))
        second = list(np.random.rand(len(self.data)))
        x = np.concatenate((np.array([one]), np.array([second])), axis=0)
        lista = []
        for el in x.transpose():
            lista.append((el[0], el[1]))
        lista.sort(key=lambda x: x[1])
        return list(map(lambda el: int(el[0]), lista))

    # główna część - algorytm uczenia się
    # BMU - indeks najsilniej odpowiadającego neuronu
    def learning(self, number_of_epochs):

        # służy do zapisywania stanów sieci w kolejnych epokach do późniejszej obserwacji
        remember_network = [copy.deepcopy(self.network), ]

        for epoch in range(number_of_epochs):
            for index in self.randomize_data_indexes():
                x = self.data[index].reshape(1, len(self.data[0]))
                s = epoch / number_of_epochs
                if (self.by_dot_product):
                    BMU = self.network.layers.find_best_neuron_by_dot_product(x)
                else:
                    BMU = self.network.layers.find_best_neuron_by_cartesian_distance(x)
                # print(BMU)
                self.nbh_fun(x, BMU, s, self.dist_0)
            print("epoch: " + str(epoch))
            print(self.network.layers.W.transpose()[0])
            remember_network.append(copy.deepcopy(self.network))
        return remember_network

    # funkcja malejąca do learning rate
    def alpha_fun(self, s, a0=1., t=2.0):
        if (s != 1):
            return a0 * np.exp(t - t / (1 - s))
        else:
            return 0.

    # funkcja ustalająca zmniejszający się rozmiar zasięgu funkcji sąsiedstwa
    def deacresing_dist_fun(self, dist0, s, t=1.2):
        return self.alpha_fun(s, dist0, t)

    # nazwa jest niedokładna. Jest to funkcja sąsiedstwa, oraz algorytm uczenia neuronu na jej podstawie:
    # x - wektor wejściowy
    # BMU - indeks neuronu najmocniej odpowiadającego
    # s - zmienna informująca o tym ile zostało do zakończenia epok
    # dist_0 - zasięg w "neuronach" początkowy - zasięg definiowany jako odchylenie standardowe funkcji Gaussa
    def nbh_fun(self, x, BMU, s, dist_0):
        layer = self.network.layers
        W = layer.W.transpose()
        dist_0_act = self.deacresing_dist_fun(dist_0, s)
        gauss = Gauss(0., dist_0_act)  # uzywamy zwyklego Gaussa1D do policzenia "wzmocnienia" w funkcji odleglosci
        for v in range(layer.number_of_neurons()):
            dist = layer.cartesian_distance_between_neurons(BMU, v)
            k = gauss.val(dist)
            W[v] = W[v] + (self.alpha_fun(s, self.a_0, 0.7) * k * (x[0] - W[v]))
